<a href="https://colab.research.google.com/github/vmartinezarias/Curso_Ecologia_Paisaje_y-Ecoacustica/blob/main/10%20-%20Antropofonias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Antropofonías

Este análisis se basa en la caracterización de Antropofonías mediante el uso de los niveles de presión sonora (SPL). El análisis consiste de tres partes: Generación de gráficas, comparación bio vs anthro, generación del excel con los datos finales.

Este algoritmo se basa en lo publicado por Ulloa et al.

https://scikit-maad.github.io/install.html


El nivel de presión sonora (SPL) es una métrica fundamental en acústica que mide la intensidad de las ondas sonoras, expresada en decibelios (dB). En ecoacústica, el SPL se utiliza para evaluar la energía acústica en un paisaje sonoro, permitiendo identificar contribuciones de fuentes como la biofonía (sonidos biológicos), antropofonía (sonidos humanos) y geofonía (sonidos ambientales). Al calcular el SPL en bandas de frecuencia específicas, se pueden analizar patrones temporales y espaciales de actividad acústica, lo que lo convierte en una herramienta clave para monitorear la biodiversidad, evaluar el impacto del ruido humano y estudiar las condiciones ambientales de un ecosistema.

El SPL es una métrica energética que permite inferir la intensidad y tipo de actividad sonora en un área. Valores altos en bandas biofónicas suelen reflejar mayor actividad biológica, mientras que SPL altos en bandas antropofónicas pueden indicar contaminación sonora. Esta métrica se usa ampliamente para monitorear la biodiversidad, caracterizar paisajes sonoros y evaluar cambios en respuesta a disturbios ambientales, convirtiéndolo en un indicador esencial en estudios de ecoacústica.

## Instalar paquetes requeridos

In [ ]:
!pip install scikit-maad

## Generación de gráficas


### Importe de paquetes y enlace con Drive

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from maad import sound, util, spl
import concurrent.futures
import time
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

### Configuraciones iniciales

In [ ]:
# Configuraciones iniciales
input_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Audios_SinLluvia'
output_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados_SPL'
batch_size = 40
prefix = "CursoEcolPasEcoac_"  # Prefijo para las gráficas

# Parámetros (los que están aquí son los de AUDIOMOTH sensu Ulloa)
S = -18
G = 16
VADC = 2

# Bandas de frecuencia
frequency_bands = [
    (0, 1000), (1000, 2000), (2000, 3000), (3000, 4000),
    (4000, 5000), (5000, 6000), (6000, 7000), (7000, 8000),
    (8000, 9000), (9000, 10000), (10000, 11000), (11000, 12000)
]

### Ejecutar algoritmo y generar gráficas SPL

In [ ]:
# Crear el directorio de salida si no existe
if not os.path.exists(output_path):
    os.makedirs(output_path)

def process_audio_file(fname):
    """Procesa un archivo de audio y calcula el SPL promedio por bandas de frecuencia."""
    try:
        wave, fs = sound.load(fname, channel='left', detrend=True)
        Sxx_power, _, fn, _ = sound.spectrogram(wave, fs, window='hann', nperseg=1024, noverlap=512 // 2)
        mean_PSD = np.mean(Sxx_power, axis=1)

        leq_bands = [
            spl.psd2leq(mean_PSD[util.index_bw(fn, band)], gain=G, sensitivity=S, Vadc=VADC)
            for band in frequency_bands
        ]

        basename = os.path.basename(fname)
        time_str = basename.split('_')[2].split('.')[0]
        hour = int(time_str[:2])

        return leq_bands, hour
    except Exception as e:
        print(f"Error processing file {fname}: {e}")
        return None

def process_batch(flist):
    """Procesa un lote de archivos en paralelo."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_audio_file, flist))
    return results

def generate_spl_for_subfolder(subfolder_path, subfolder_name):
    """Genera SPL promedio por hora para una subcarpeta y guarda una gráfica de calor."""
    flist = glob.glob(subfolder_path + '*.WAV')
    flist.sort()

    spl_data = {hour: [] for hour in range(24)}

    for i in range(0, len(flist), batch_size):
        batch_files = flist[i:i + batch_size]
        batch_results = process_batch(batch_files)

        for result in batch_results:
            if result is not None:
                leq_bands, hour = result
                spl_data[hour].append(leq_bands)

    hourly_avg_spl = {
        hour: np.nanmean(values, axis=0).tolist() if values else [np.nan] * len(frequency_bands)
        for hour, values in spl_data.items()
    }

    # Crear un DataFrame con los resultados
    df_hourly_avg_spl = pd.DataFrame(hourly_avg_spl, index=[f"{fmin}-{fmax}Hz" for fmin, fmax in frequency_bands])
    df_hourly_avg_spl = df_hourly_avg_spl.iloc[::-1]
    df_hourly_avg_spl.replace([None, []], np.nan, inplace=True)

    # Graficar el heatmap
    plt.figure(figsize=(15, 5))
    sns.heatmap(df_hourly_avg_spl.astype(float), cmap='RdPu', annot=True, fmt=".1f")
    plt.title(f'{prefix}{subfolder_name} - Promedio de nivel de presión sonora (dB SPL) por banda de frecuencia')
    plt.xlabel('Hora del día')
    plt.ylabel('Banda de frecuencia (Hz)')
    plt.xticks(ticks=np.arange(24), labels=[f"{hour}:00" for hour in range(24)])
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Guardar la gráfica en el directorio de salida
    output_file = os.path.join(output_path, f'{prefix}_{subfolder_name}_SPL_heatmap_S-18G16.png')
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfica guardada en: {output_file}")
    return df_hourly_avg_spl

if __name__ == "__main__":
    start_time = time.time()
    for root, dirs, _ in os.walk(input_path):
        for dir in dirs:
            subfolder_path = os.path.join(root, dir) + '/'
            generate_spl_for_subfolder(subfolder_path, dir)
    print(f"Tiempo de ejecución: {time.time() - start_time} segundos.")


## Comparación bandas Bio vs Antro

### Carga de paquetes

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from maad import sound, util, spl
import concurrent.futures
import time
# Montar Google Drive
drive.mount('/content/drive')

### Configuraciones iniciales

In [ ]:
# Configuraciones iniciales
input_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Audios_SinLluvia'
output_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados_SPL'
batch_size = 40
prefix = "CursoEcolPasEcoac_"  # Prefijo para las gráficas

# Parámetros (los que están aquí son los de AUDIOMOTH sensu Ulloa)
S = -18
G = 16
VADC = 2

# Bandas de frecuencia
frequency_bands = [
    (0, 1000), (1000, 2000), (2000, 3000), (3000, 4000),
    (4000, 5000), (5000, 6000), (6000, 7000), (7000, 8000),
    (8000, 9000), (9000, 10000), (10000, 11000), (11000, 12000)
]

### Ejecución del algoritmo

In [ ]:
# Crear el directorio de salida si no existe
if not os.path.exists(output_path):
    os.makedirs(output_path)

def process_audio_file(fname):
    """Procesa un archivo WAV y calcula los SPL por banda de frecuencia con manejo de errores."""
    try:
        wave, fs = sound.load(fname, channel='left', detrend=True)
        Sxx_power, _, fn, _ = sound.spectrogram(wave, fs, window='hann', nperseg=1024, noverlap=512 // 2)
        mean_PSD = np.mean(Sxx_power, axis=1)

        leq_bands = [
            spl.psd2leq(mean_PSD[util.index_bw(fn, band)], gain=G, sensitivity=S, Vadc=VADC)
            for band in frequency_bands
        ]

        basename = os.path.basename(fname)
        time_str = basename.split('_')[2].split('.')[0]
        hour = int(time_str[:2])

        return leq_bands, hour
    except Exception as e:
        print(f"Error processing file {fname}: {e}")
        return None

def process_batch(flist):
    """Procesa un lote de archivos en paralelo."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_audio_file, flist))
    return results

def generate_spl_for_subfolder(subfolder_path, subfolder_name):
    """Genera SPL promedio por hora para una subcarpeta y crea gráficos comparativos."""
    flist = glob.glob(subfolder_path + '*.WAV')
    flist.sort()

    spl_data = {hour: [] for hour in range(24)}

    for i in range(0, len(flist), batch_size):
        batch_files = flist[i:i + batch_size]
        batch_results = process_batch(batch_files)

        for result in batch_results:
            if result is not None:
                leq_bands, hour = result
                spl_data[hour].append(leq_bands)

    hourly_avg_spl = {
        hour: np.nanmean(values, axis=0).tolist() if values else [np.nan] * len(frequency_bands)
        for hour, values in spl_data.items()
    }

    # Crear un DataFrame con los resultados
    df_hourly_avg_spl = pd.DataFrame(hourly_avg_spl, index=[f"{fmin}-{fmax}Hz" for fmin, fmax in frequency_bands])
    df_hourly_avg_spl = df_hourly_avg_spl.iloc[::-1]

    # Calcular Antropofonía (0-1000 Hz) y Biofonía (media de 1000-12000 Hz)
    anthropophony = df_hourly_avg_spl.loc['0-1000Hz'].astype(float)
    biophony = df_hourly_avg_spl.loc['1000-2000Hz':].astype(float).mean()

    # Crear gráfica comparativa
    plt.figure(figsize=[7, 4])
    plt.plot(anthropophony.index, anthropophony.values, label='Antropofonía')
    plt.plot(biophony.index, biophony.values, label='Biofonía')
    plt.xlabel('Hora')
    plt.ylabel('Nivel de presión sonora (dB SPL)')
    plt.legend()
    plt.title(f'{prefix}{subfolder_name} - Comparación de SPL entre bandas de Antropofonía y Biofonía')
    plt.tight_layout()

    # Guardar la gráfica
    output_file = os.path.join(output_path, f'{prefix}_{subfolder_name}_Anthropophony_vs_Biophony_S-18G16.png')
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfica guardada en: {output_file}")
    return df_hourly_avg_spl

if __name__ == "__main__":
    start_time = time.time()
    for root, dirs, _ in os.walk(input_path):
        for dir in dirs:
            subfolder_path = os.path.join(root, dir) + '/'
            generate_spl_for_subfolder(subfolder_path, dir)
    print(f"Tiempo de ejecución: {time.time() - start_time} segundos.")

## Generación de Excel con Datos

### Carga de paquetes

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import concurrent.futures
from maad import sound, util, spl
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

### Configuraciones iniciales

In [ ]:
# Configuraciones iniciales
input_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Audios_SinLluvia'
output_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados_SPL'
batch_size = 40
prefix = "CursoEcolPasEcoac_"  # Prefijo para las gráficas

# Parámetros (los que están aquí son los de AUDIOMOTH sensu Ulloa)
S = -18
G = 16
VADC = 2

# Bandas de frecuencia
frequency_bands = [
    (0, 1000), (1000, 2000), (2000, 3000), (3000, 4000),
    (4000, 5000), (5000, 6000), (6000, 7000), (7000, 8000),
    (8000, 9000), (9000, 10000), (10000, 11000), (11000, 12000)
]

### Generación del excel

In [ ]:
# Crear el directorio de salida si no existe
if not os.path.exists(output_path):
    os.makedirs(output_path)

def process_audio_file(fname):
    """Procesa un archivo WAV y calcula los SPL por banda de frecuencia con manejo de errores."""
    try:
        wave, fs = sound.load(fname, channel='left', detrend=True)
        Sxx_power, _, fn, _ = sound.spectrogram(wave, fs, window='hann', nperseg=1024, noverlap=512 // 2)
        mean_PSD = np.mean(Sxx_power, axis=1)

        leq_bands = [
            spl.psd2leq(mean_PSD[util.index_bw(fn, band)], gain=G, sensitivity=S, Vadc=VADC)
            for band in frequency_bands
        ]

        basename = os.path.basename(fname)
        time_str = basename.split('_')[2].split('.')[0]
        hour = int(time_str[:2])
        folder_name = os.path.basename(os.path.dirname(fname))

        return leq_bands, hour, folder_name
    except Exception as e:
        print(f"Error processing file {fname}: {e}")
        return None

def process_batch(flist):
    """Procesa un lote de archivos en paralelo."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_audio_file, flist))
    return results

def compile_results_for_all_folders(input_path, prefix, excel_writer):
    """Compila los resultados SPL de todas las carpetas en una sola hoja de Excel manejando archivos erróneos."""
    compiled_results = []

    for root, dirs, _ in os.walk(input_path):
        for dir in dirs:
            subfolder_path = os.path.join(root, dir) + '/'
            flist = glob.glob(subfolder_path + '*.WAV')
            flist.sort()

            if not flist:
                continue

            batch_results = process_batch(flist)

            for result in batch_results:
                if result is not None:
                    leq_bands, hour, folder_name = result
                    compiled_results.append([prefix + folder_name, hour] + leq_bands)

    columns = ['Folder', 'Hour'] + [f"{fmin}-{fmax}Hz" for fmin, fmax in frequency_bands]
    df_compiled_results = pd.DataFrame(compiled_results, columns=columns)
    df_compiled_results.to_excel(excel_writer, sheet_name='Compiled Results', index=False, float_format="%.2f")

if __name__ == "__main__":
    import time

    start_time = time.time()

    # Crear un escritor de pandas Excel
    excel_writer = pd.ExcelWriter(os.path.join(output_path, f'{prefix}SPL_Summary.xlsx'), engine='openpyxl')

    # Compilar y guardar resultados de todas las carpetas
    compile_results_for_all_folders(input_path, prefix, excel_writer)

    # Guardar y cerrar el archivo Excel
    excel_writer.close()
    print(f"Tiempo de ejecución: {time.time() - start_time} segundos.")

# Generación de tablas para intepolación

## Cargar Paquetes

In [ ]:
import pandas as pd
import numpy as np
import os
from openpyxl import Workbook

## Definir parámetros

In [ ]:

# Ruta del archivo Excel de entrada
input_file_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados_SPL/CursoEcolPasEcoac_SPL_Summary.xlsx'

# Parámetros definidos manualmente
hora_inicio = 17  # Hora de inicio del rango
hora_fin = 7      # Hora final del rango
freq_inicio = "0-1000Hz"  # Frecuencia inicial
freq_fin = "3000-4000Hz"  # Frecuencia final
output_file_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados_SPL/CursoEcolPasEcoac_promedios_SPL.xlsx'


## Correr algoritmos

In [ ]:

# Leer el archivo Excel
print(f"Leyendo el archivo Excel: {input_file_path}")
datos = pd.read_excel(input_file_path)

# Asegurarse de que las columnas de frecuencia estén presentes
if freq_inicio not in datos.columns or freq_fin not in datos.columns:
    raise ValueError("Frecuencia inicial o final no encontrada en las columnas del archivo.")

# Determinar las columnas de frecuencia dentro del rango especificado
rango_frecuencias = datos.columns.tolist()
col_freq_inicio = rango_frecuencias.index(freq_inicio)
col_freq_fin = rango_frecuencias.index(freq_fin)
if col_freq_inicio > col_freq_fin:
    raise ValueError("La columna de frecuencia inicial está después de la final en las columnas del archivo.")

# Filtrar las horas manejando rangos que cruzan la medianoche
if hora_inicio <= hora_fin:
    datos_filtrados = datos[(datos['Hour'] >= hora_inicio) & (datos['Hour'] <= hora_fin)]
else:
    datos_filtrados = datos[(datos['Hour'] >= hora_inicio) | (datos['Hour'] <= hora_fin)]

# Seleccionar columnas relevantes: Carpeta, Hora y Rango de Frecuencias
datos_filtrados = datos_filtrados.iloc[:, [0, 1] + list(range(col_freq_inicio, col_freq_fin + 1))]

# Calcular promedios agrupados por 'Folder'
data_promedios = datos_filtrados.groupby('Folder').mean(numeric_only=True)

# Guardar los resultados en un archivo Excel
print(f"Guardando el archivo Excel: {output_file_path}")
data_promedios.to_excel(output_file_path, index=True, engine='openpyxl')
print(f"Archivo guardado con éxito en: {output_file_path}")

# Mostrar una vista previa de los resultados
print("Vista previa de los promedios calculados:")
print(data_promedios.head())
